In [3]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
from openai import OpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search
from dotenv import load_dotenv

load_dotenv()


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))




In [114]:
import pandas as pd
import tiktoken
import openai

from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

# from utils.embeddings_utils import get_embedding

embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

# def get_embedding(model: str, text: str) -> list[float]:
#     result = openai.Embedding.create(
#       model = model,
#       input = text
#     )
#     return result['data'][0]['embedding']

# print(get_embedding("text-embedding-3-small", ))


vectorDB = {}

import nltk


files = ["Admin Worker_ SOP document.txt", "Employee Communications and Techniques (Team Conflicts) - SOP and Guidelines.txt", "Leadership Development_ SOP and Guidelines.txt", "Software Engineer_ SOP and Code Documentation.txt"]
for i in files:
    with open(f"./sops/{i}", "r") as file:
        file = file.read()
        count = 0
        chunk = 100
        chunks = []
        print(len(encoding.encode(file)))

        response = client.embeddings.create(
                input=i,
                model="text-embedding-3-small"
        )
        chunks.append(response.data[0].embedding)

        # while True:
        #     print(count)
        #     if count > len(file) - 1: break
        #     chunking = file[count:chunk+count]
        #     print(chunking)
        #     response = client.embeddings.create(
        #         input=chunking,
        #         model="text-embedding-3-small"
        #     )
        #     chunks.append(response.data[0].embedding)
        #     count += chunk



        x = []
        sent_text = nltk.sent_tokenize(file) # this gives us a list of sentences
        for i in sent_text:
            x += [l.strip() for l in i.split("\n") if l]

        for k in x:
            response = client.embeddings.create(
                    input=k,
                    model="text-embedding-3-large"
            )
            chunks.append(response.data[0].embedding)

        vectorDB[i] = chunks
        print(len(chunks))


import json
with open('vectorDB.json', 'w') as f:
    json.dump(vectorDB, f)




[0.005132982973009348, 0.017242904752492905, -0.018698474392294884, -0.01855851523578167, -0.047250036150217056, -0.03028704971075058, 0.027669822797179222, 0.003614431945607066, 0.011217685416340828, 0.006434598471969366, -0.0016865016659721732, 0.01581532694399357, -0.0012849952327087522, -0.00785867776721716, 0.05990229919552803, 0.050357118248939514, -0.02754385955631733, 0.00991606991738081, -0.040392063558101654, 0.04999322444200516, -0.0003859796852339059, 0.030231066048145294, -0.013729942962527275, 0.032974254339933395, 0.01731288433074951, 0.01680903322994709, -0.0017661030869930983, 0.020391974598169327, 0.040811941027641296, -0.03770485892891884, -0.02617226541042328, -0.0500212162733078, 0.024184852838516235, -0.05519968643784523, -0.03224647045135498, 0.04232349246740341, 0.06466089189052582, 0.014695657417178154, -0.015619384124875069, -0.04134378209710121, 0.02218344435095787, 0.007361824624240398, 0.04495471343398094, 0.007074909284710884, -0.024072885513305664, 0.0523

In [71]:
import pandas as pd 

df = None

with open('vectorDB.json', 'r') as f:
    vectorDB = json.load(f)
    df = pd.DataFrame(columns=["fileName", "embedding"])
    for i in vectorDB.items():
        df.loc[len(df.index)] = list(i)

print(df)


                                            fileName  \
0                     Admin Worker_ SOP document.txt   
1  Employee Communications and Techniques (Team C...   
2     Leadership Development_ SOP and Guidelines.txt   
3  Software Engineer_ SOP and Code Documentation.txt   

                                           embedding  
0  [[0.004638994112610817, 0.04118556156754494, 0...  
1  [[-0.022587936371564865, 0.031012626364827156,...  
2  [[-0.01195149403065443, -0.004546619486063719,...  
3  [[-0.005015118513256311, 0.024436600506305695,...  


In [115]:
# from openai.embeddings_utils import get_embedding, cosine_similarity

from typing import List, Optional
from scipy import spatial
import numpy as np



from scipy.spatial.distance import cosine


# df["distances"] = df["embeddings"].apply(lambda x: cosine(q_embeddings, x))

def calculate_similarity(x, embedding):
   print([cosine(i, embedding) for i in x])
   return np.max([cosine(i, embedding) for i in x])

def search(df, search_description, n=3, pprint=True):
   embedding = client.embeddings.create(
            input=search_description,
            model="text-embedding-3-small"
        )
   
   df['similarities'] = df.embedding.apply(lambda x: calculate_similarity(x, embedding.data[0].embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

res = search(df, 'document management', n=3)

print(res)


[0.5441320065911462, 0.5578060915286589, 0.6250476951766297, 0.6466890212052497, 0.5474985534763657, 0.6392529241799241, 0.6845832408807404, 0.702842902330403, 0.2822275697278821, 0.4705652277716509, 0.506849461124683, 0.6857924787701505, 0.7113823264886117, 0.7451563173730849, 0.6602460664107263, 0.728668275244638, 0.6677612163812001, 0.7482318388645002, 0.6987530936458368, 0.7846651424959681, 0.679373774004377, 0.6093957658396549, 0.7900996191515693, 0.6163633939026811, 0.7545235171303089, 0.7427067967146064, 0.8299098348377371, 0.7214231556069275, 0.577456515653109, 0.5908430423787134, 0.5320825760081772]
[0.6553279511593255, 0.6304677195781022, 0.7194209750883895, 0.747932857722142, 0.7501823161721233, 0.7967859755810094, 0.7496817164639407, 0.7473631478652814, 0.8067914986817775, 0.7805256227867979, 0.8331769769967843, 0.8153680620200269, 0.803440990226258, 0.7792472765239332, 0.7878795505717847, 0.7684662613759424, 0.6200801281616368, 0.6428671390800955, 0.7800039487532758, 0.796

In [31]:
df.embedding[0].shape

AttributeError: 'list' object has no attribute 'shape'

In [122]:
import nltk

a = [[0.5441320065911462, 0.5578060915286589, 0.6250476951766297, 0.6466890212052497, 0.5474985534763657, 0.6392529241799241, 0.6845832408807404, 0.702842902330403, 0.2822275697278821, 0.4705652277716509, 0.506849461124683, 0.6857924787701505, 0.7113823264886117, 0.7451563173730849, 0.6602460664107263, 0.728668275244638, 0.6677612163812001, 0.7482318388645002, 0.6987530936458368, 0.7846651424959681, 0.679373774004377, 0.6093957658396549, 0.7900996191515693, 0.6163633939026811, 0.7545235171303089, 0.7427067967146064, 0.8299098348377371, 0.7214231556069275, 0.577456515653109, 0.5908430423787134, 0.5320825760081772]
,[0.6553279511593255, 0.6304677195781022, 0.7194209750883895, 0.747932857722142, 0.7501823161721233, 0.7967859755810094, 0.7496817164639407, 0.7473631478652814, 0.8067914986817775, 0.7805256227867979, 0.8331769769967843, 0.8153680620200269, 0.803440990226258, 0.7792472765239332, 0.7878795505717847, 0.7684662613759424, 0.6200801281616368, 0.6428671390800955, 0.7800039487532758, 0.7968869274015621, 0.7304400349938265, 0.6773758396265788, 0.6751964970094629]
,[0.6716156564411679, 0.6358633895592328, 0.8280963660427623, 0.8049936665848267, 0.8690029989188381, 0.8292189496864879, 0.7759864440903608, 0.899168414674286, 0.8153804066474146, 0.8799310484694859, 0.7231085608619204, 0.7495841279021902, 0.802469982071769, 0.8316649477988749, 0.824405252398113, 0.8225579898076385, 0.800140765494697, 0.8410274347044049, 0.6261822593909036, 0.5854241002370721, 0.6934825799780151]
,[0.6190239634073829, 0.7075365368294738, 0.704950499297862, 0.813659521383784, 0.7339325929438005, 0.7756940278164377, 0.7455027692678655, 0.5657557136724394, 0.6946967810005877, 0.7319472607249445, 0.7783902016097959, 0.7719100984182595, 0.6049186444897456, 0.7721716530881498, 0.7846859233454708, 0.5764961798409294, 0.561296172643927, 0.5621164104296799, 0.6866271990456478, 0.733334975551915, 0.6081057080037489, 0.739130326578789, 0.7547599257735786, 0.902980082359319, 0.9448373999946501, 0.952916807994488, 0.937844586953957, 0.9511405984506922, 0.8886173160945999, 0.8488891896125396, 0.8928012270989327, 0.8345712567757511, 0.7169523006059886]]

c = [np.argmax(i) for i in a]
print(c)

files = ["Admin Worker_ SOP document.txt", "Employee Communications and Techniques (Team Conflicts) - SOP and Guidelines.txt", "Leadership Development_ SOP and Guidelines.txt", "Software Engineer_ SOP and Code Documentation.txt"]
for j,i in enumerate(files):
    with open(f"./sops/{i}", "r") as file:
        file = file.read()
        x = []
        sent_text = nltk.sent_tokenize(file) # this gives us a list of sentences
        for i in sent_text:
            x += [l.strip() for l in i.split("\n") if l]
        print(x[c[j]])


# # now loop over each sentence and tokenize it separately
# for sentence in sent_text:
#     tokenized_text = nltk.word_tokenize(sentence)
#     tagged = nltk.pos_tag(tokenized_text)
#     print(tagged)

[26, 10, 7, 25]
* Maintain electronic and paper filing systems.
Responsibilities:
* Approved by: [Supervisor/Manager's Name]
4.


In [111]:
a = [[0.5441320065911462, 0.5578060915286589, 0.6250476951766297, 0.6466890212052497, 0.5474985534763657, 0.6392529241799241, 0.6845832408807404, 0.702842902330403, 0.2822275697278821, 0.4705652277716509, 0.506849461124683, 0.6857924787701505, 0.7113823264886117, 0.7451563173730849, 0.6602460664107263, 0.728668275244638, 0.6677612163812001, 0.7482318388645002, 0.6987530936458368, 0.7846651424959681, 0.679373774004377, 0.6093957658396549, 0.7900996191515693, 0.6163633939026811, 0.7545235171303089, 0.7427067967146064, 0.8299098348377371, 0.7214231556069275, 0.577456515653109, 0.5908430423787134, 0.5320825760081772]
,[0.6553279511593255, 0.6304677195781022, 0.7194209750883895, 0.747932857722142, 0.7501823161721233, 0.7967859755810094, 0.7496817164639407, 0.7473631478652814, 0.8067914986817775, 0.7805256227867979, 0.8331769769967843, 0.8153680620200269, 0.803440990226258, 0.7792472765239332, 0.7878795505717847, 0.7684662613759424, 0.6200801281616368, 0.6428671390800955, 0.7800039487532758, 0.7968869274015621, 0.7304400349938265, 0.6773758396265788, 0.6751964970094629]
,[0.6716156564411679, 0.6358633895592328, 0.8280963660427623, 0.8049936665848267, 0.8690029989188381, 0.8292189496864879, 0.7759864440903608, 0.899168414674286, 0.8153804066474146, 0.8799310484694859, 0.7231085608619204, 0.7495841279021902, 0.802469982071769, 0.8316649477988749, 0.824405252398113, 0.8225579898076385, 0.800140765494697, 0.8410274347044049, 0.6261822593909036, 0.5854241002370721, 0.6934825799780151]
,[0.6190239634073829, 0.7075365368294738, 0.704950499297862, 0.813659521383784, 0.7339325929438005, 0.7756940278164377, 0.7455027692678655, 0.5657557136724394, 0.6946967810005877, 0.7319472607249445, 0.7783902016097959, 0.7719100984182595, 0.6049186444897456, 0.7721716530881498, 0.7846859233454708, 0.5764961798409294, 0.561296172643927, 0.5621164104296799, 0.6866271990456478, 0.733334975551915, 0.6081057080037489, 0.739130326578789, 0.7547599257735786, 0.902980082359319, 0.9448373999946501, 0.952916807994488, 0.937844586953957, 0.9511405984506922, 0.8886173160945999, 0.8488891896125396, 0.8928012270989327, 0.8345712567757511, 0.7169523006059886]
]

c = [np.argmax(i) for i in a]
c

[26, 10, 7, 25]

In [110]:
x = "4."
y = "* Maintain electronic and paper filing systems."
# y = "Document Management:"
z = "document management"

a = [x,y,z]
b = []
for i in a:
    embedding = client.embeddings.create(
                input=i,
                model="text-embedding-3-large"
            )
    b.append(embedding.data[0].embedding)
print(b)

print(cosine(b[0],b[1]))

print(cosine(b[0],b[2]))


[[0.01291978731751442, -0.004515989683568478, -0.013117639347910881, 0.02445460483431816, 0.016382209956645966, -0.014918100088834763, -0.030568255111575127, 0.13762639462947845, -0.00035335271968506277, 0.06398559361696243, 0.001880838070064783, 0.008324655704200268, -0.0103477006778121, -0.0016792755341157317, -0.018123315647244453, 0.014927992597222328, -0.026512272655963898, 0.031102458015084267, -0.040282826870679855, -0.022199083119630814, 0.015828223899006844, 0.024553531780838966, -0.008250460959970951, 0.038838502019643784, 0.014759818091988564, 0.02423696592450142, 0.026611199602484703, 0.01683727279305458, -0.0016075539169833064, 0.022891568019986153, 0.027679605409502983, 0.023128990083932877, 0.015452302992343903, -0.025028377771377563, -0.007231519557535648, 0.004513516556471586, 0.05342025309801102, -0.004768252372741699, -0.02283221110701561, 0.013424311764538288, 0.02020077034831047, -0.030568255111575127, -0.0019525596871972084, 0.03632577136158943, 0.0714248567819595

In [125]:
embedding = client.embeddings.create(
                input=i,
                model="text-embedding-3-large"
            )
x = embedding.data[0].embedding
print(x)
print(df.embedding[3][25])

cosine(df.embedding[3][25], x)

[-0.027647467330098152, 0.00476045161485672, -0.015535356476902962, 0.0010734788374975324, -0.024889852851629257, 0.031237121671438217, -0.010703586973249912, 0.01913689635694027, -0.028693459928035736, 0.003218207973986864, 7.740005821688101e-05, -0.024723444133996964, -0.02147849276661873, 0.0024768018629401922, 0.0020102655980736017, 0.004097791854292154, -0.026007162407040596, 0.013692984357476234, 0.03499318286776543, -0.028194235637784004, 0.024747217074036598, 0.003108259988948703, 0.0573393739759922, 0.005108125042170286, 0.019457826390862465, -0.022037146613001823, 0.00808266457170248, 0.03829756751656532, 0.022738438099622726, -0.021014926955103874, 0.02187073975801468, -0.006989127490669489, 0.009045452810823917, 0.007856825366616249, 0.041815903037786484, 0.006067941430956125, -0.005803471896797419, 0.002952252747491002, -0.022821640595793724, -0.0020533534698188305, 0.027457287535071373, -0.0077439057640731335, 0.026815427467226982, 0.015761196613311768, -0.037750799208879

ValueError: shapes (1536,) and (3072,) not aligned: 1536 (dim 0) != 3072 (dim 0)

In [179]:
from openai import OpenAI
client = OpenAI()


career = '''
Open-ended: What are your long-term career goals, and how do you see your role here at the Port of Singapore Authority contributing to those goals?
Answer: My long-term career goal is to work in a leadership position within the logistics and operations sector. I’m particularly interested in improving the efficiency of supply chain processes and adopting new technologies to enhance port productivity. My current role at the Port of Singapore Authority allows me to gain valuable hands-on experience in day-to-day operations, while also giving me insights into the challenges and opportunities of large-scale logistics management. This experience is helping me build the foundational skills necessary for a future leadership role.

Multiple Choice: Which of the following areas are you most interested in pursuing further in your career?
a) Operations and Logistics
b) Management and Leadership
c) Technology and Innovation
d) Customer Relations and Services
e) Other: [Please specify]
Answer: B

Open-ended: Can you describe a project or role you have enjoyed working on the most, and why it was meaningful to you?
Answer: One of the most enjoyable projects I’ve worked on was implementing a new cargo tracking system. The project involved coordinating with various departments, testing the technology, and training staff on how to use the system. I found it meaningful because it significantly improved the efficiency of our cargo handling processes and reduced delays. The collaboration required across departments also helped me develop better teamwork and communication skills.

Multiple Choice: How would you rate your interest in taking on leadership roles in the future?
a) Very interested
b) Somewhat interested
c) Neutral
d) Not very interested
e) Not interested at all
Answer: A

Open-ended: What skills or experiences do you feel you need to develop further to achieve your desired career pathway within the Port of Singapore Authority?
Answer: I believe I need to further develop my strategic decision-making skills, particularly in risk management and financial forecasting. I also want to gain more experience in cross-departmental coordination, as well as exposure to advanced technologies used in port automation and logistics. Additionally, I think improving my data analytics skills would help me make better-informed decisions as I pursue a leadership role in the future.
'''

quiz = '''Quiz: Administrative SOP at PSA
Section 1: General Procedures
What is the purpose of the Standard Operating Procedure (SOP) for Administrative Tasks?
a) To track financial reports
b) To provide standardized processes for administrative tasks
c) To ensure legal compliance
d) To automate administrative workflows
Who is responsible for assisting with technical issues related to administrative tasks?
a) Department Heads
b) Administrative Assistants
c) IT Support
d) PSA Management
What is the designated review frequency of the SOP?
a) Monthly
b) Annually
c) Bi-annually
d) As needed when IT changes occur

Section 2: Correspondence & Document Management
How should incoming mail be managed?
a) Send all mail directly to the department head
b) Keep incoming mail for 2 days before distribution
c) Log it in the mail logbook and distribute it promptly
d) Scan and email all mail to recipients
What is the deadline for sending all outgoing mail?
a) By noon each business day
b) By the end of each business day
c) Within 48 hours
d) By the next working day
What should administrative staff do with documents no longer needed according to PSA's document retention policy?
a) Keep them indefinitely for auditing purposes
b) Archive them off-site immediately
c) Conduct periodic purges as necessary
d) Shred all documents after one month

Section 3: Meeting & Travel Arrangements
Within how many hours should meeting minutes be distributed?
a) 8 hours
b) 12 hours
c) 24 hours
d) 48 hours
What must be arranged before scheduling a meeting?
a) Catering for all participants
b) Relevant agendas and confirmation of attendees
c) External guest approvals
d) Travel reimbursement forms
What document is provided to travelers when booking travel?
a) Travel reimbursement form
b) Travel budget report
c) Itinerary and necessary travel documents
d) Insurance policy

Section 4: Inventory & Health and Safety
What should be done if office supplies reach the minimum threshold?
a) Send a warning email to all employees
b) Notify IT support
c) Reorder the supplies immediately
d) Report to the finance department
Who is responsible for conducting regular safety drills?
a) Administrative Assistants
b) Health and Safety Officer
c) IT Support
d) Department Head
What is the correct action if a staff member encounters malfunctioning equipment?
a) Wait for maintenance to inspect it
b) Report it to the relevant department and log it
c) Discard the equipment immediately
d) Inform other staff to avoid using it

Section 5: Compliance and Record-Keeping
What must be done with administrative records to ensure compliance?
a) Share them publicly
b) Ensure they are available for audits and reviews
c) Store only electronic copies
d) Delete them after each quarter
Who must approve revisions to the SOP?
a) Administrative Assistants
b) IT Support
c) Department Heads
d) External auditors
When should revisions be made to the SOP?
a) Only if the department head requests changes
b) Annually or when changes in procedures or personnel occur
c) Every month
d) Only during audit reviews

Answer Key:
b, 2) c, 3) b, 4) c, 5) b, 6) c, 7) c, 8) b, 9) c, 10) c, 11) b, 12) b, 13) b, 14) c, 15) b

'''

data = []
files = ["Admin Worker_ SOP document.txt", "Employee Communications and Techniques (Team Conflicts) - SOP and Guidelines.txt", "Leadership Development_ SOP and Guidelines.txt", "Software Engineer_ SOP and Code Documentation.txt"]
for i in files:
    with open(f"./sops/{i}", "r") as file:
        file = file.read()
        data.append(file)


completion = client.chat.completions.create(
  response_format={ "type": "json_object" },
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f'''
      Given these context below:
     {data}

    You would be given a quiz results, come up with areas where the skills are lacking. rank it in order of more severely lacking to least severly lacking. return the answer in the following json format: 
      <area of improvement>: <description>

    do not return anything else other than the dictionary, keep the json keys short and conceise.
    '''},
    {"role": "user", "content": quiz}
  ]
)

skill_gap = json.loads(completion.choices[0].message.content)

completion = client.chat.completions.create(
  response_format={ "type": "json_object" },
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f'''
      you are a HR manager at Port of Singapore Authority

    You would be given a questionaire answers answered by one of your employees, come up with some insights into the person's career aspirations. return the answer in the json format with the following keys:
     career_aspirations, long_term_goals, primary_interests, leadership_interest, skill_development, current_job_scope

    do not return anything else other than the dictionary.
    '''},
    {"role": "user", "content": career}
  ]
)

insights = json.loads(completion.choices[0].message.content)


print(skill_gap, insights)


def generate_case_study(insights, skill_gaps):

  completion = client.chat.completions.create(
    response_format={ "type": "json_object" },
    model="gpt-4o",
    messages=[
      {"role": "system", "content": f'''
      given the following context/company's SOPs and documents: {data}

      you are a talent training personnel at Port of Singapore Authority. you are tasked with generating a unique and personalised case study based on your target's career aspirations and skill gaps identified. 
      your case study generated should be able to aim to do the following things: 
      1) help the target to improve on his skill gaps (eg: if he has been identified to be lacking in "Inventory & Health and Safety", then the case study should be testing him on the relevant Inventory & Health and Safety procedures)
      2) identify what he needs to work in to attain his career aspirations, and ensure that the case study allows him to learn those skills which would be required for him to achieve his goals. (eg: if he has aspirations to work in a leadership position, then the case study should test him on his leadership skills, etc) 
      3) the case study should be based on a real-world scenario (but adapt the scenario a little to make it unique). the case study should have an objective. write the scenarios and tasks in coherent complete sentences. give more description for the scenarios for the target to analyse.
      4) for the tasks generated, generate a overall task for them to do, write in complete sentences in a single paragraph.
      5) the case study should not be just asking the person to list out what he knows or do not know, it should be allowing him to demonstrate his learnings through a actual scenario that a person with the same role should know how to do. 
      6) use the given company's SOPs and documents to generate the case study
      7) be specific in the scenario generated, do not give too generic scenarios. 

      the case study should be engaging, realistic, and short enough to be completed not more than a few hours.

      the target would do the case study with a mentor (who is more senior), you are also required to generate a list of follow up or prompting questions for the mentor to ask the target during the case study session. 
      these questions should be inquisitive and invite more conversations between the mentor and the target. it should allow the target to learn from the experiences of the mentor. 
      
      Return the answer in the json format with the following keys:
      case_study (title, scenario, step_by_step_tasks, overall_task), follow_up_questions 

      do not return anything else other than the dictionary.
      '''},
      {"role": "user", "content": f'''
      career aspirations: {insights}

      skill gaps: {list(skill_gap.values())}
      '''}
    ]
  )

  return json.loads(completion.choices[0].message.content)

{'General Procedures': 'Purpose of SOP misunderstood; many chose answers related to financial or legal compliance instead of standardized processes.', 'Correspondence Management': 'Handling of incoming mail not clear; sending to department head was a common misconception instead of logging and prompt distribution.', 'Document Management': 'Misinterpretation around document retention policy, with errors on periodic purges necessity.', 'Meeting Arrangements': "Confusion about prerequisites for scheduling meetings; participants didn't realize agendas and confirmation of attendees are primary.", 'Inventory Management': 'Uncertainty in action to take when supplies hit minimum threshold; many failed to prioritize immediate reorder.', 'Compliance': 'Misunderstanding on ensuring records are available for audits as some thought public sharing was acceptable.'} {'career_aspirations': 'To work in a leadership position within the logistics and operations sector, with a focus on improving supply ch

In [184]:
print(insights)
print(skill_gap)

{'career_aspirations': 'To work in a leadership position within the logistics and operations sector, with a focus on improving supply chain efficiency and integrating new technologies.', 'long_term_goals': 'To build foundational skills and eventually assume a leadership role in logistics management, enhancing port productivity through innovative solutions.', 'primary_interests': 'Management and Leadership', 'leadership_interest': 'Very interested', 'skill_development': 'Strategic decision-making, risk management, financial forecasting, cross-departmental coordination, experience with advanced technologies in port automation, and data analytics.', 'current_job_scope': 'Gaining hands-on experience in day-to-day operations and insights into the challenges and opportunities in logistics management.'}
{'General Procedures': 'Purpose of SOP misunderstood; many chose answers related to financial or legal compliance instead of standardized processes.', 'Correspondence Management': 'Handling of

In [183]:
case_study = generate_case_study(insights, skill_gap)

print(case_study['case_study']['scenario'])
print(case_study['case_study']['overall_task'])
print(case_study['case_study']['step_by_step_tasks'])


In this case study, you are an aspiring logistics manager at PSA, keen on leading a team to streamline supply chain operations. Recently, the port has been experiencing inefficiencies due to outdated administrative processes and a lack of coordination between departments. To address these issues, the senior management has decided to implement a new strategic initiative. You have been assigned to lead this project. You must review current administrative practices, propose improvements, and ensure that they align with the strategic goals of integrating new technologies and enhancing supply chain efficiency. Your mentor, an experienced logistics leader, will review your proposed strategies to ensure they meet PSA's operational standards.
Your task is to develop a comprehensive administrative and leadership strategy aimed at addressing the inefficiencies within the port's supply chain operations. Present this strategy to the senior mentor, demonstrating how it aligns with PSA's strategic g

In [164]:
import requests
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt=f'''
        Generate a realistic picture which shows the following case scenario. Do not include any words in the photo. Do not include any annotations in the photo. 
        
        case scenario: {case_study['case_study']['scenario']}
    ''',
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

image_response = requests.get(image_url)
if image_response.status_code == 200:
    with open('image_01.png', 'wb') as file:
        file.write(image_response.content)
        print("Image saved successfully!")
else:
    print(f"Failed to download image: {image_response.status_code}")

Image saved successfully!


In [165]:
case_study['follow_up_questions']

['What strategies did you employ to engage stakeholders during the project initiation meeting, and how did it impact the overall project alignment?',
 "How did you ensure compliance with the company's SOPs when assessing inventory management practices and documenting your findings?",
 'Can you discuss a specific challenge you encountered in integrating new technology with existing systems, and how you addressed it?',
 'What role did data analytics play in your decision-making process while developing the project plan?',
 'How did you balance risk management considerations with the potential benefits of adopting the new inventory management software?',
 'In what ways did collaboration with cross-departmental stakeholders contribute to the success of your project proposal?']

In [176]:
transcript = '''
Person: I've been preparing for the team meeting where I’ll lead the discussion on integrating new technology into our logistics processes. It’s a big responsibility since my supervisor is away, and I want to ensure that everything runs smoothly.
Mentor: That’s a great opportunity for you to step up. What strategies are you considering to engage the stakeholders during the meeting and ensure that everyone aligns with the project goals?
Person: I plan to start with a clear agenda and emphasize the potential benefits of the new technology—like how it can improve supply chain efficiency and reduce manual errors. I’ll also encourage each department to share their input on how the integration might impact their workflows, making them feel involved in the process.
Mentor: Sounds good. Engaging stakeholders is crucial. And how did you ensure that your approach aligns with PSA’s SOPs, particularly when assessing the current inventory management practices?
Person: I’ve been reviewing the relevant SOPs to make sure that our assessment process is thorough and compliant. I’ll document everything we discuss during the meeting, especially when it comes to safety and operational standards, so that we have a proper record to refer back to.
Mentor: Excellent. Documentation is key, especially for compliance. Let’s talk about the integration itself—did you encounter any challenges when trying to merge the new technology with our existing systems? How did you handle them?
Person: Yes, there was a challenge with ensuring compatibility between the new software and our current system. We realized that some data formats weren’t aligning, which could have led to data inconsistencies. To address this, I worked closely with the IT team to develop a custom data conversion process. It took a bit of back and forth, but we managed to make the systems communicate seamlessly.
Mentor: That’s a good approach, leveraging internal expertise. Another aspect to consider—how did you use data analytics in your decision-making process while you were developing the project plan?
Person: Data analytics was critical. I analyzed historical data on our supply chain performance to identify key bottlenecks and areas where the new technology could make the biggest impact. It helped me prioritize which features of the new system we should focus on implementing first, so we could see quick wins and build momentum for the larger integration.
Mentor: Smart move, focusing on data to guide the process. Lastly, I’m curious—how did you manage the balance between risk and the potential benefits when proposing this new inventory management software?
Person: I conducted a risk assessment with the team, looking into potential disruptions during the transition phase, like delays or staff needing extra training time. But I also highlighted how the long-term benefits, such as improved tracking and faster turnaround times, outweighed those initial challenges. I presented this balanced view during the meeting to ensure we were making an informed decision.
Mentor: You’ve thought through this thoroughly, which is great to see. Remember to maintain that balance and keep the lines of communication open, especially with the cross-departmental stakeholders. Their buy-in can make a significant difference in the success of the integration.
'''

transcript2 = '''
Person: I've started working on my plan to enhance the administrative efficiency of logistics operations. My focus has been on streamlining document management and optimizing the mail handling process. I think these are crucial areas that could benefit from technological advancements.
Mentor: That’s a good start! Let's dive deeper into some aspects. For example, how do you see technology impacting your proposed changes in administrative tasks and logistics operations?
Person: I believe that digital integration is key to improving the overall workflow. For document management, using cloud-based solutions can help ensure that all relevant documents are accessible in real time. For mail handling, an automated sorting system could reduce manual sorting time, speeding up the entire process.
Mentor: That makes sense. With digital integration, you can definitely boost efficiency. But what strategic decision-making skills did you employ while addressing inefficiencies in the mail handling process?
Person: I conducted a process audit to identify bottlenecks, like delays in manual sorting and inaccuracies in correspondence tracking. Based on this analysis, I decided that automation would be the most effective way to reduce errors and processing time. I also prioritized training staff on new systems to ensure a smooth transition.
Mentor: Sounds like you’re thinking through it carefully. Another aspect to consider is how cross-departmental coordination can enhance operational efficiency. What are your thoughts on that?
Person: Cross-departmental coordination is crucial, especially between logistics and administration. For example, if the administrative team is aligned with the needs of the logistics department, we can better anticipate the required documentation and ensure that everything is in order before shipments. This reduces downtime and avoids last-minute scrambling for paperwork, which can cause delays.
Mentor: Absolutely. It’s important to think beyond your immediate scope. How would you align your proposal with the organization’s long-term goals of improving supply chain efficiency?
Person: My plan is designed to support the broader objective of a more responsive and agile supply chain. By automating and streamlining administrative processes, we can ensure that the logistics team has accurate, real-time information, reducing bottlenecks in the supply chain. This also ties into improving safety compliance and ensuring a quicker response to any issues that arise.
Mentor: Great connection to the bigger picture! Now, when it comes to monitoring the impact of these changes, how do you plan to do that after your proposal is approved?
Person: I would set up key performance indicators (KPIs) such as document turnaround time, error rates in correspondence, and time taken to complete safety drills. Regular feedback sessions with staff would also be helpful to assess how well the new processes are being adopted. Additionally, monthly reports could be used to track progress and make adjustments as needed.
Mentor: It sounds like you’ve put a lot of thought into this, which is important for building your leadership profile. One last thing—what are some key leadership qualities you think you’ve demonstrated while working on this case study?
Person: I think my analytical approach to identifying process inefficiencies shows problem-solving skills. My ability to visualize the integration of technology into operations reflects my strategic thinking. And by planning for staff training and seeking feedback, I’ve tried to emphasize collaborative leadership. These qualities are what I hope will prepare me for a full leadership role in logistics and operations.
Mentor: Those are great insights. Keep refining your plan, and don’t forget to stay adaptable. It’s all about balancing the immediate needs with the long-term vision. You’re on the right track!
'''

completion = client.chat.completions.create(
  response_format={ "type": "json_object" },
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f'''
     given the following context/company's SOPs and documents: {data}

    you are a talent training personnel at Port of Singapore Authority. 
    you are given a transcript of a person's conversation with his mentor, in relation to doing a case study. you are also given the case study itself.
    you are tasked with evaluating any skills gaps the person have
    you aim to do the following things: 
    1) come up with areas where there are skills that are lacking. 
    2) rank it in order of more severely lacking to least severly lacking. 
    3) use the given company's SOPs and documents  
    
    
    return the answer in the following json format: 
      <area of improvement>: <description>

    do not return anything else other than the dictionary, keep the json keys short and conceise.
    '''},
    {"role": "user", "content": f'''
    case_study: {case_study}

    transcript: {transcript2}
    '''}
  ]
)

skill_gap = json.loads(completion.choices[0].message.content)
skill_gap

{'tech_implementation': "Insufficient detail on how to effectively implement proposed technologies and integrations as per company's SOP.",
 'strategic_decision_making': 'Lack of strategic planning regarding prioritization and phased implementation of automation measures.',
 'cross_departmental_coordination': 'Needs improvement in detailed planning for interdepartmental collaboration to boost operational efficiency.',
 'kpi_establishment': 'Limited specifics on establishing and measuring key performance indicators to validate the impact of changes.',
 'change_management': "While training is considered, lacks a comprehensive change management plan for the staff's transition to new systems.",
 'safety_compliance_procedures': 'Insufficient emphasis on conducting safety drills and documenting safety compliance effectively.'}

In [177]:
# def generate_case_study(insights, skill_gaps):

#   completion = client.chat.completions.create(
#     response_format={ "type": "json_object" },
#     model="gpt-4o",
#     messages=[
#       {"role": "system", "content": f'''
#       given the following context/company's SOPs and documents: {data}

#       you are a talent training personnel at Port of Singapore Authority. you are tasked with generating a unique and personalised case study based on your target's career aspirations and skill gaps identified. 
#       your case study generated should be able to aim to do the following things: 
#       1) help the target to improve on his skill gaps (eg: if he has been identified to be lacking in "Inventory & Health and Safety", then the case study should be testing him on the relevant Inventory & Health and Safety procedures)
#       2) identify what he needs to work in to attain his career aspirations, and ensure that the case study allows him to learn those skills which would be required for him to achieve his goals. (eg: if he has aspirations to work in a leadership position, then the case study should test him on his leadership skills, etc) 
#       3) the case study should be based on a real-world scenario (but adapt the scenario a little to make it unique). the case study should have an objective. write the scenarios and tasks in coherent complete sentences. give more description for the scenarios for the target to analyse.
#       4) for the tasks generated, generate a overall task for them to do, write in complete sentences in a single paragraph.
#       5) the case study should not be just asking the person to list out what he knows or do not know, it should be allowing him to demonstrate his learnings through a actual scenario that a person with the same role should know how to do. 
#       6) use the given company's SOPs and documents to generate the case study

#       the target would do the case study with a mentor (who is more senior), you are also required to generate a list of follow up or prompting questions for the mentor to ask the target during the case study session. 
#       these questions should be inquisitive and invite more conversations between the mentor and the target. it should allow the target to learn from the experiences of the mentor. 
      
#       Return the answer in the json format with the following keys:
#       case_study (title, scenario, step_by_step_tasks, overall_task), follow_up_questions 

#       do not return anything else other than the dictionary.
#       '''},
#       {"role": "user", "content": f'''
#       career aspirations: {insights}

#       skill gaps: {list(skill_gap.values())}
#       '''}
#     ]
#   )

  # return json.loads(completion.choices[0].message.content)

case_study2 = generate_case_study(insights, skill_gap)
case_study2

{'case_study': {'title': 'Strategic Leadership in Logistics and Operations: Implementing Technology for Enhanced Efficiency',
  'scenario': "You are an aspiring logistics leader at the Port of Singapore Authority. Your manager has tasked you with leading a project aimed at implementing new automated sorting systems to increase efficiency in cargo handling. This initiative aligns with the company's objective to modernize the port's operations while improving safety and interdepartmental collaboration. The project requires a thorough understanding of the company's current SOPs, strategic planning for technology adoption, and comprehensive planning for change management. Your manager expects you to make strategic decisions on technology implementation phases, detail planning for departmental coordination, and establish key performance indicators (KPIs) to evaluate the impact of the changes.",
  'step_by_step_tasks': ['Conduct a gap analysis based on current operations and identify areas w

In [ ]:
case_study2['case_study']['title']